In [2]:
# all of the imports
import pandas as pd
import numpy as np
import pickle 
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.metrics import SCORERS
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV



% matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
#Raw file
dfs=pd.read_csv('https://www.dropbox.com/s/dws32wrufaccumd/cars1_stats.csv?dl=1',sep='\t')

In [7]:
cdata = dfs.iloc[:,1:]

In [8]:
cdata

,Model,Price,Miles,Location,Distance,VIN,Mpg,Transmission,Engine,DriveType,FuelType,Trim,CarFax,Exterior,Interior
0,2017 Hyundai Sonata SE 2.0L Hybrid,"18,888","24,788",San Jose,- 2 mi.,KMHE24L33HA061101,Unknown,Automatic,Gas/Electric I-4 2.0 L,FWD,Plug-in,SE 2.0L Hybrid,Carfax® Report,Nocturne Black,Gray
1,2017 Kia Forte LX Sedan Automatic,"10,872","39,823",Santa Clara,- 1 mi.,3KPFL4A7XHE009022,29 cty / 38 hwy,Automatic,4 Cyl 2.0L Gas,FWD,Gas,LX Sedan Automatic,Carfax® Report,Silky Silver,Black
2,2017 Chevrolet Malibu LT with 1LT,"14,772","40,694",Santa Clara,- 1 mi.,1G1ZE5ST0HF187948,27 cty / 36 hwy,Automatic,Turbocharged Gas I4 1.5L,FWD,Gas,LT with 1LT,Carfax® Report,Nightfall Gray Metallic,Jet Black
3,2017 Chevrolet Malibu LT with 1LT,"14,772","43,010",Santa Clara,- 1 mi.,1G1ZE5ST5HF188562,27 cty / 36 hwy,Automatic,Turbocharged Gas I4 1.5L,FWD,Gas,LT with 1LT,Carfax® Report,Nightfall Gray Metallic,Jet Black
4,2017 Chevrolet Malibu LT with 1LT,"14,772","43,829",Santa Clara,- 1 mi.,1G1ZE5ST9HF188676,27 cty / 36 hwy,Automatic,Turbocharged Gas I4 1.5L,FWD,Gas,LT with 1LT,Carfax® Report,Summit White,Jet Black
5,2006 Honda Civic Sedan Hybrid CVT,"7,950","68,359",Walnut Creek,- 41 mi.,JHMFA36276S014636,49 cty / 51 hwy,Automatic,4 Cyl 1.3L Gas/Electric Hybrid,FWD,Hybrid,Sedan Hybrid CVT,Carfax® Report,Gray,Blue
6,2016 Mitsubishi Outlander Sport 2.0 ES FWD CVT,"13,995","64,389",San Jose,- 2 mi.,JA4AP3AU0GZ033857,24 cty / 30 hwy,Automatic,4 Cyl 2.0L Gas,FWD,Gas,2.0 ES FWD CVT,Carfax® Report,Red Metallic,Black
7,2017 Chevrolet Malibu LT with 1LT,"15,072","38,238",Oakland,- 15 mi.,1G1ZE5ST4HF190111,27 cty / 36 hwy,Automatic,Turbocharged Gas I4 1.5L,FWD,Gas,LT with 1LT,Carfax® Report,Nightfall Gray Metallic,Jet Black
8,2017 Chevrolet Malibu LT with 1LT,"14,772","40,241",San Francisco,- 25 mi.,1G1ZE5ST1HF186601,27 cty / 36 hwy,Automatic,Turbocharged Gas I4 1.5L,FWD,Gas,LT with 1LT,Carfax® Report,Nightfall Gray Metallic,Jet Black
9,2008 Land Rover LR2 HSE,"11,995","74,138",Hayward,- 24 mi.,SALFT24N78H075450,16 cty / 23 hwy,Automatic,Inline 6 3.2,AWD,Gas,HSE,Carfax® Report,Charcoal,Unknown


In [9]:
cdata.columns

Index(['Model', 'Price', 'Miles', 'Location', 'Distance', 'VIN', 'Mpg',
       'Transmission', 'Engine', 'DriveType', 'FuelType', 'Trim', 'CarFax',
       'Exterior', 'Interior'],
      dtype='object')

In [22]:
cdata[(cdata.Mpg=='Unknown')]

,Model,Price,Miles,Location,Distance,VIN,Mpg,Transmission,Engine,DriveType,FuelType,Trim,CarFax,Exterior,Interior,citympg,hwympg,Year,Make


In [ ]:
cdata=cdata.dropna()




In [11]:
cdata['citympg'] = cdata.apply(lambda row: row['Mpg'].split('/')[0].replace('cty','').strip(), axis=1)

In [14]:
hwympg=[]
for i in cdata['Mpg']:
    if 'Unknown' in i:
        hwympg.append('Unknown')
    else:
        hwympg.append(i.split('/')[1].replace('hwy','').strip())
cdata['hwympg']=hwympg

In [15]:
cdata['Price']=cdata.apply(lambda row:row['Price'].replace(',','').strip(),axis=1)

In [16]:
cdata['Miles']=cdata.apply(lambda row:row['Miles'].replace(',','').strip(),axis=1)

In [17]:
cdata['Distance']=cdata.apply(lambda row:row['Distance'].replace('- ','').replace(' mi.','').strip(),axis=1)

In [18]:
year=[]
make=[]
model=[]
for i in cdata['Model']:
    list_str = i.split(' ')
    year.append(list_str[0])
    make.append(list_str[1])
    model.append((" ").join(list_str[2:]))
cdata['Year']=year
cdata['Make']=make
cdata['Model']=model




In [19]:
sub1=['Price','Miles','Distance','citympg','hwympg']

for i in sub1:
    cdata[i] = cdata[i].replace('Unknown',np.nan).astype(float)




In [24]:
cdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2384 entries, 1 to 2399
Data columns (total 19 columns):
Model           2384 non-null object
Price           2384 non-null float64
Miles           2384 non-null float64
Location        2384 non-null object
Distance        2384 non-null float64
VIN             2384 non-null object
Mpg             2384 non-null object
Transmission    2384 non-null object
Engine          2384 non-null object
DriveType       2384 non-null object
FuelType        2384 non-null object
Trim            2384 non-null object
CarFax          2384 non-null object
Exterior        2384 non-null object
Interior        2384 non-null object
citympg         2384 non-null float64
hwympg          2384 non-null float64
Year            2384 non-null object
Make            2384 non-null object
dtypes: float64(5), object(14)
memory usage: 372.5+ KB


In [25]:
cdata['Year'] = cdata.Year.astype(int)

In [26]:
cdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2384 entries, 1 to 2399
Data columns (total 19 columns):
Model           2384 non-null object
Price           2384 non-null float64
Miles           2384 non-null float64
Location        2384 non-null object
Distance        2384 non-null float64
VIN             2384 non-null object
Mpg             2384 non-null object
Transmission    2384 non-null object
Engine          2384 non-null object
DriveType       2384 non-null object
FuelType        2384 non-null object
Trim            2384 non-null object
CarFax          2384 non-null object
Exterior        2384 non-null object
Interior        2384 non-null object
citympg         2384 non-null float64
hwympg          2384 non-null float64
Year            2384 non-null int64
Make            2384 non-null object
dtypes: float64(5), int64(1), object(13)
memory usage: 372.5+ KB


In [27]:
#Making column age for the cars
cdata['Age']=2018 - cdata.Year 

In [28]:
#Replacing 4WD with AWD
cdata['DriveType']=cdata.apply(lambda row:row['DriveType'].replace('4WD','AWD').strip(),axis=1)